# Terminals Shapefiles

In [2]:
%load_ext autoreload
%autoreload
import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>
    div.output_area{
        max-height:10000px;
        overflow:scroll;
    }
</style>"""))

import shapefile
import pygeohash as geohash
from timeUtils import clock, elapsed
from shapely.geometry.polygon import Polygon
from shapely.geometry import Point
from random import uniform
from os.path import join
import pickle
from glob import glob
import string
from os.path import basename,splitext
from collections import Counter
from geoUtils import *

Notebook Last Run Initiated: 2018-12-12 18:40:32.976684


# Global Parameters

In [ ]:
# Global Params
prec=7
basedir="/Users/tgadf/Downloads/terminals"

# Process Geo Data

In [3]:
from os.path import basename, dirname, join
shapeval = "Airports"
try:
    sf = shapefile.Reader(join(basedir, shapeval, shapeval))
except:
    raise ValueError("No shapefile!")
fields      = sf.fields
shapeData   = {}
geoShapeMap = {}
Nshapes   = len(sf.shapes())
ngeos     = 0
totalgeos = 0
prec      = 8
show      = False

start,cmt = clock("Analyzing {0}\t{1}".format(shapeval, Nshapes))
if show:
    print("\n\nFields -> {0}".format(fields))


iname=None
for i,val in enumerate(fields):
    if val[0] == "FullName":
        iname = i-1
        break
itype=None
for i,val in enumerate(fields):
    if val[0] == "OwnerType":
        itype = i-1
        break
iacres=None
for i,val in enumerate(fields):
    if val[0] == "Acres":
        iacres = i-1
        break

irec = -1
from numpy import sqrt
from haversine import haversine
from random import uniform
for shapeRec in sf.iterShapeRecords():
    irec += 1
    if irec % 1000 == 0:
        print(irec)

    ## Record
    record = shapeRec.record
    if show: raise ValueError("Stopping here: {0}".format(record))
    geoid = record[1]
    name  = record[iname]
    size  = record[iacres]
    atype = record[itype]
    shapeData[geoid] = {"Name": name, "Record": irec, "Type": atype, "Size": size}

    geos = addLinearGeos(irec, Nshapes, shapeRec.shape, prec, debug=False)

    if size is not None:
        newgeos = set()
        rad = 63.6167*sqrt(size)/2
        long, lat = shapeRec.shape.points[0]
        nmiss = 0
        for i in range(10000):
            dLg = uniform(-rad, rad) / 111000
            dLa = uniform(-rad, rad) / 111000
            pnt = (lat + dLa, long + dLg)
            dist = haversine((lat, long), pnt)
            if dist < rad/1000:
                geo    = geohash.encode(latitude=pnt[0], longitude=pnt[1], precision=7)
                if geo not in newgeos:
                    newgeos.add(geo)
                    nmiss = 0
                else:
                    nmiss += 1

            if nmiss > 200:
                break
            
        geoShapeMap[geoid] = newgeos
        ngeos += len(newgeos)
    else:
        geoShapeMap[geoid] = geos
        ngeos += len(geos)
        
print("Found {0} geos from {1}".format(ngeos, shapeval))
saveGeoData(shapeData, geoShapeMap, Nshapes, ngeos, "terminals/{0}-{1}".format(shapeval, 7))

Current Time is Wed Dec 12, 2018 18:40:34 for Analyzing Airports	19460
0


NameError: name 'addLinearGeos' is not defined

In [3]:
from os.path import basename, dirname
shapeval = "Amtrak_Stations"
try:
    sf = shapefile.Reader(join(basedir, shapeval, shapeval))
except:
    raise ValueError("No shapefile!")
fields      = sf.fields
shapeData   = {}
geoShapeMap = {}
Nshapes   = len(sf.shapes())
ngeos     = 0
totalgeos = 0
prec      = 7
show      = False

start,cmt = clock("Analyzing {0}\t{1}".format(shapeval, Nshapes))
if show:
    print("\n\nFields -> {0}".format(fields))


irec = -1
for shapeRec in sf.iterShapeRecords():
    irec += 1

    ## Record
    record = shapeRec.record
    if show: raise ValueError("Stopping here: {0}".format(record))
    geoid = record[1]
    name  = record[2]
    shapeData[geoid] = {"Name": name, "Record": irec}
        
    geos = addLinearGeos(irec, Nshapes, shapeRec.shape, prec, debug=False)

    
    if len(geos) > 0:
        newgeos = set()
        rad = 100
        long, lat = shapeRec.shape.points[0]
        nmiss = 0
        for i in range(10000):
            dLg = uniform(-rad, rad) / 111000
            dLa = uniform(-rad, rad) / 111000
            pnt = (lat + dLa, long + dLg)
            dist = haversine((lat, long), pnt)
            if dist < rad/1000:
                geo    = geohash.encode(latitude=pnt[0], longitude=pnt[1], precision=7)
                if geo not in newgeos:
                    newgeos.add(geo)
                    nmiss = 0
                else:
                    nmiss += 1

            if nmiss > 200:
                break

        geoShapeMap[geoid] = newgeos
        ngeos += len(newgeos)

print("Found {0} geos from {1}".format(ngeos, shapeval))
saveGeoData(shapeData, geoShapeMap, Nshapes, ngeos, "terminals/{0}-{1}".format(shapeval, 7))

Current Time is Wed Sep 19, 2018 20:27:47 for Analyzing Amtrak_Stations	529
Found 2575 geos from Amtrak_Stations


There are 529 entries in the saved file.
Saved shape data to terminals/Amtrak_Stations-7-data.p


There are 2575 entries in the saved file.
Saved shape data to terminals/Amtrak_Stations-7-geos.p




# Combine

In [4]:

vals     = glob(join(basedir, "*-{0}-geos.p".format(prec)))
basevals = set([splitext(basename(x))[0].split('-')[0] for x in vals])
print(vals)
geomap   = {}
records  = {}
ngeos    = 0
totalgeos = 0
for baseval in basevals:
    try:
        recname = "{0}/{1}-{2}-data.p".format(basedir, baseval, prec)
        recdata = pickle.load(open(recname, "rb"))
    except:
        print("Could not open {0}".format(recname))
        continue
    try:
        geoname = "{0}/{1}-{2}-geos.p".format(basedir, baseval, prec)
        geodata = pickle.load(open(geoname, "rb"))
    except:
        print("Could not open {0}".format(geoname))
        continue
    ngeos = 0
    
    if baseval.startswith("Amtrak"):
        baseval="Amtrak"
    elif baseval.startswith("Airport"):
        baseval="Airport"

    for geoid,geos in geodata.items():
        ngeos += len(geos)
        totalgeos += ngeos
        for geo in geos:
            if geomap.get(geo) is None:
                geomap[geo] = Counter()
            geomap[geo][baseval] += 1

    for geoid,rec in recdata.items():
        records[geoid] = baseval #rec['Name']
        #print(rec['Name'])
        break

    print("{0}\t{1}\t---> {2} <---".format(baseval,ngeos,len(geomap)))


for geo in geomap.keys():
    geomap[geo] = dict(geomap[geo])
            
print("Found {0} geos in total".format(totalgeos))
fname=join(basedir, "geomap-{0}-Terminals.p".format(prec))
print("Writing {0}".format(fname))
pickle.dump(geomap,  open(fname, "wb"))

fname=join(basedir, "georec-{0}-Terminals.p".format(prec))
print("Writing {0}".format(fname))
pickle.dump(records,  open(fname, "wb"))

['/Users/tgadf/Downloads/terminals/Airports-7-geos.p', '/Users/tgadf/Downloads/terminals/Amtrak_Stations-7-geos.p']
Airport	507037	---> 506462 <---
Amtrak	2575	---> 509020 <---
Found 5269546172 geos in total
Writing /Users/tgadf/Downloads/terminals/geomap-7-Terminals.p
Writing /Users/tgadf/Downloads/terminals/georec-7-Terminals.p
